In [6]:
from playwright.async_api import async_playwright
import asyncio
import nest_asyncio

nest_asyncio.apply()

# Explore jina.ia to get the extracted text: https://jina.ai/reader#demo

async def extract_linkedin_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        
        print(f"Navigating to {url}")
        await page.goto(url)
        await page.wait_for_timeout(3000)  # Wait for 3 seconds

        print("Extracting visible text")
        text = await page.evaluate("() => document.body.innerText")

        print(f"Extracted {len(text)} characters of text")
        
        await browser.close()
        return text

# Usage
linkedin_url = "https://www.linkedin.com/company/hyertekinc"
text_content = asyncio.get_event_loop().run_until_complete(extract_linkedin_details(linkedin_url))
print(text_content + "...")  # Print first 200 characters

Navigating to https://www.linkedin.com/company/hyertekinc
Extracting visible text
Extracted 7122 characters of text
Skip to main content
LinkedIn
Articles
People
Learning
Jobs
Games
Join now
Sign in
HyerTek
IT Services and IT Consulting
Rockville, Maryland  1,871 followers
See jobs
Follow
  

View all 11 employees

About us

HyerTek provides expert technology solutions for federal government clients, with a strong emphasis on innovation, integration, and automation that help enhance citizens lives. 



Founded in 2016, HyerTek, specializes in providing innovative and customized technology solutions. Our business expertise is firmly planted in the areas of  data analytics and business intelligence, cloud computing, collaboration, human centered design, and technical training. 

Our team is dedicated to understanding the unique needs and goals of each project, and we approach every task with enthusiasm and care. We are skilled in developing solutions that not only meet the technical requ